In [9]:
import json
r = None
with open("mcmd2_result_5_cht.txt", "r", encoding='utf-8') as file:
    r = json.load(file)

    
    
b = {}

for k,v in r.items():
    b[k] = v['im_gateways2']
    
b

{'0.2': ['778638', '778671', '778806', '778650', '778860'],
 '0.4': ['778638', '778671', '778806', '778650', '778860'],
 '0.6000000000000001': ['778638', '778671', '778650', '778737', '778784'],
 '0.8': ['778638', '778671', '778650', '778737', '778784'],
 '1.0': ['778638', '778671', '778650', '778737', '778784']}

In [ ]:
{'0.2': ['778638', '778671', '778806', '778650', '778860'],
 '0.4': ['778638', '778671', '778806', '778650', '778860'],
 '0.6': ['778638', '778671', '778650', '778737', '778784'],
 '0.8': ['778638', '778671', '778650', '778737', '778784'],
 '1.0': ['778638', '778671', '778650', '778737', '778784']}

In [ ]:
t = 0
g = 0
for k,v in route_subgraphs.items():
    if nx.is_strongly_connected(v):
        t += 1
    g +=1
    
t

In [ ]:
# for k,v in route_subgraphs.items():
#     n = [node for node, out_degree in v.out_degree() if out_degree == 0]
#     m = [node for node, in_degree in v.in_degree() if in_degree == 0]
#     #n = r.out_degree()
#     print(n)
#     print(m)
#     print()

In [ ]:
# import matplotlib.pyplot as plt

# p = time_table.groupby(['route_id', 'direction_id'])['stop_id'].apply(lambda x: x.tolist()).to_dict()
# #dep_per_route = dep_list.groupby('route_id')['deps'].apply(lambda x: x.tolist()).to_dict(into=OrderedDict)

# for r in all_routes:
#     p0 = [namify_stop(G.name, s) for s in p.get((r, 0), [])]
#     p1 = [namify_stop(G.name, s) for s in p.get((r, 1), [])]
#     g0 = G.subgraph(p0).copy()
#     g1 = G.subgraph(p1).copy()
#     #nx.draw_spectral(g0)
#     #plt.show()
#     n = [node for node, out_degree in g0.out_degree() if out_degree == 0]
#     if len (g0) != 0:
#         print(len(n), 'n', r)
#         if len(n) == 0:
#             nx.draw_spectral(g0)
#             plt.show()
#     m = [node for node, out_degree in g1.out_degree() if out_degree == 0]
#     if len (g1) != 0:
#         print(len(m), 'm', r)
#         if len(m) == 0:
#             nx.draw_spectral(g1)
#             plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# t = trip_subgraphs['t_76016_b_17595_tn_1']
# nx.draw(t, with_labels = True)
# plt.show

In [ ]:
# for r in all_routes:
#         time_table
#     break
    
# r
#     #0_sub_nodes = [namify_stop(G.name, s) for s in stops_per_route[r]]
#     # G.remove_nodes_from([n for n in G if n not in set(nodes)])
#     #sub_graph = G.subgraph(sub_nodes).copy()
#     #route_subgraphs[r] = sub_graph

In [ ]:
def generate_sensor_scenerios1(set_count, min_count, max_count):
    global G, routes_per_stop # input

    sensor_scenerios = []
    all_stops = [get_stopid(s) for s in set(G.nodes)]
    
    for i in range(set_count):
        sensor_count = random.randint(min_count, max_count)
        all_sensors = np.random.choice(all_stops, size=sensor_count, replace=False)
        
        sensor_objects = {}

        msg_gen_rate = np.random.randint(low = sim.msg_gen_rate_range[0], high= sim.msg_gen_rate_range[1], size=sensor_count) # 10mins to 12 hours
        start_time = np.random.randint(low = sim.msg_gen_rate_range[0], high=sim.msg_gen_rate_range[1], size=sensor_count) # 0 to 1 hour
        np.random.shuffle(start_time)

        for i,sensor_name in enumerate(all_sensors):
            r = routes_per_stop[get_stopid(sensor_name)]

            s = OnRouteSensor(name=sensor_name, routes=r, start_time=start_time[i], msg_gen_rate=msg_gen_rate[i], msg_ttl=None, data_size=None)
            sensor_objects[sensor_name]=s
        sensor_scenerios.append(sensor_objects)
        
    return sensor_scenerios

def get_sensor_score(sensor):
    return 1/sensor.msg_gen_rate

def generate_scenerios_route_scores(scenerios):
    global stops_per_route
    
    scenerios_scores_per_route = []
    for sensor_objects in scenerios:
        score_per_route = {}
        for r in stops_per_route.keys():
            score = 0
            sensors = stops_per_route[r].intersection(set(sensor_objects.keys()))
            score = sum([get_sensor_score(sensor_objects[s]) for s in sensors])
#             print(sensor_objects.keys())
#             for s in sensors:
#                 print(get_sensor_score(sensor_objects[s]))
#                 return
            score_per_route[r] = score
        
        scenerios_scores_per_route.append(score_per_route)
    
    return scenerios_scores_per_route
    

def local_optima2(p_gateway, routes_per_stop, routes_covered, cost_per_stop):
    global scenerios,scenerios_route_scores
    value = 0
    
    stops_served = set(e for s in (routes_per_stop[p_gateway] - routes_covered) for e in s)
    for i,sensor_objects in enumerate(scenerios):
        p_gateway_sensors = stops_served.intersection(set(sensor_objects.keys()))
        for sensor in p_gateway_sensors:
            value += get_sensor_score(sensor)
                
    return value/cost_per_stop.get(p_gateway, 1)

def local_optima(p_gateway, routes_per_stop, routes_covered, cost_per_stop):
    global scenerios,scenerios_route_scores
    value = 0
    
    routes_served = routes_per_stop[p_gateway] - routes_covered
    for i,_ in enumerate(scenerios):
        for route in routes_served:
            value += scenerios_route_scores[i][route]
                
    return value/cost_per_stop.get(p_gateway, 1)


scenerios = generate_sensor_scenerios1(1, 15, 20)
scenerios_route_scores = generate_scenerios_route_scores(scenerios)
set_cover_greedy_gateway_selection(all_routes, routes_per_stop)

In [ ]:
from heapdict import heapdict
import copy
import time


class Node(object):
    def __init__(self, node):
        self.node = node
        self.mg1 = 0
        self.prev_best = None
        self.mg2 = 0
        self.flag = None
        self.list_index = 0


def celfpp_v2(G, budget, n_scenerios, min_sensor_count=10, max_sensor_count=30):
    start_time = time.time()
    scenerios = generate_sensor_scenerios(n_scenerios, min_sensor_count, max_sensor_count)

    graph = copy.deepcopy(G)
    S = set()
    # Note that heapdict is min heap and hence add negative priorities for
    # it to work.
    Q = heapdict()
    last_seed = None
    cur_best = None
    node_data_list = []

    for node in graph.nodes():
        node_data = Node(node)
        delay = compute_delay(graph, [node], scenerios)
        node_data.mg1 = sim.upper_bound_delay - delay
        # node_data.mg1 = diffuse.diffuse_mc([node])
        node_data.prev_best = cur_best
        node_data.mg2 = compute_delay(graph, [node, cur_best.node], scenerios) if cur_best else node_data.mg1
        node_data.flag = 0
        cur_best = cur_best if cur_best and cur_best.mg1 > node_data.mg1 else node_data
        graph.nodes[node]['node_data'] = node_data
        node_data_list.append(node_data)
        node_data.list_index = len(node_data_list) - 1
        Q[node_data.list_index] = - node_data.mg1

    # record the number of times the spread is computed
    lookups = [graph.number_of_nodes()]
    elapsed = [round(time.time() - start_time, 3)]

    while len(S) < budget:
        node_idx, _ = Q.peekitem()
        node_data = node_data_list[node_idx]
        if node_data.flag == len(S):
            S.add(node_data.node)
            del Q[node_idx]
            last_seed = node_data
            continue
        elif node_data.prev_best == last_seed:
            node_data.mg1 = node_data.mg2
        else:
            # before = diffuse.diffuse_mc(S)
            delay = compute_delay(graph, S, scenerios)
            before = sim.upper_bound_delay - delay
            S.add(node_data.node)
            delay = compute_delay(graph, S, scenerios)
            after = sim.upper_bound_delay - delay
            S.remove(node_data.node)
            node_data.mg1 = after - before
            node_data.prev_best = cur_best
            S.add(cur_best.node)
            delay = compute_delay(graph, S, scenerios)
            before = sim.upper_bound_delay - delay
            S.add(node_data.node)
            delay = compute_delay(graph, S, scenerios)
            after = sim.upper_bound_delay - delay
            S.remove(cur_best.node)
            if node_data.node != cur_best.node: S.remove(node_data.node)
            node_data.mg2 = after - before

        if cur_best and cur_best.mg1 < node_data.mg1:
            cur_best = node_data

        node_data.flag = len(S)
        Q[node_idx] = - node_data.mg1

    return S, elapsed, lookups

In [ ]:
import heapq
import time

def celfpp_im(graph, budget, n_scenerios, min_sensor_count=10, max_sensor_count=30):
    """
    A CELF++ algorithms for finding proper seed set with a predefined size to gain a 
        maximum influence of a network graph.
       seeds: the set containing all seeds.
       graph: the graph object.
       size_seed: the size of chosen seed set.
       mode: str, "IC" or "LT"
    """
    start_time = time.time() # in seconds
    scenerios = generate_sensor_scenerios(n_scenerios, min_sensor_count, max_sensor_count)

    best_inverse_delay = float('-inf')
    gateways = []

    pq = [[1, -1, -1, -1]] # node is an array, the first is the marginal value, the second is the node index and
                        # last is the size of temporary seed set.
    # initialization
    for node in set(graph.nodes):
        delay = compute_delay(graph, [node], scenerios)
        delay_gain = sim.upper_bound_delay - delay
        heapq.heappush(pq, [-delay_gain, node, 0, delay])

    node = heapq.heappop(pq)
    gateways.append(node[1])
    best_inverse_delay = -node[0]
    #best_delay = delay_gain
    #delay_gains = [delay_gain]
    delays = [node[3]]


    # record the number of times the spread is computed
    lookups = [graph.number_of_nodes()]
    elapsed = [round(time.time() - start_time, 3)]

    while len(gateways) < budget:
        node_lookup = 0
        while True:
            node_lookup += 1

            #tmp_set = set(gateways)
            node = heapq.heappop(pq)
            if node[2] == len(gateways):
                gateways.append(node[1])
                delays.append(node[3])
                lookups.append(node_lookup)
                elapse = round(time.time() - start_time, 3)
                elapsed.append(elapse)
                break

            #tmp_set.add(node[1])
            #diff_spread = cal_spread(graph, tmp_set)
            # diff_spread = compute_delay(graph, tmp_set, scenerios)
            delay = compute_delay(graph, gateways + [node[1]], scenerios)
            diff_spread = sim.upper_bound_delay - delay

            top = pq[0]
            if diff_spread - best_inverse_delay >= -top[0]:
                gateways.append(node[1])
                delays.append(delay)
                best_inverse_delay = diff_spread
                lookups.append(node_lookup)
                elapse = round(time.time() - start_time, 3)
                elapsed.append(elapse)
                #delay_gains.append(delay_gain)
                if len(gateways) == budget:
                    break
            else:
                node[0] = -(diff_spread - best_inverse_delay)
                node[2] = len(gateways)
                node[3] = delay
                heapq.heappush(pq, node)

    return gateways, delays, elapsed, lookups

In [ ]:
def generate_sensor_scenerios(set_count, min_sensors, max_sensors):
    import random
    global G, routes_per_stop, all_routes  # input
    #global sensor_scenerios #output
    
    #TODO:: seed works only if called from within where it is set
    #sim.seed = 0
    np.random.seed(sim.seed)
    random.seed(sim.seed)

    sensor_scenerios = []
    all_stops = [s for s in set(G.nodes)]

    for _ in range(set_count):
        sensor_count = random.randint(min_sensors, max_sensors)
        scenerio = Scenerio(graph=G, 
                            all_stops= all_stops,
                            all_routes= all_routes,
                            routes_per_stop=routes_per_stop,
                            sensor_count=sensor_count
                            )
        
        sensor_scenerios.append(scenerio)
        
    return sensor_scenerios
        
#generate_sensor_scenerios(2, 20, 50)

import heapq
import time

def celf_im(graph, budget, n_scenerios, min_sensor_count=10, max_sensor_count=30):
    """
    Find k nodes with the largest spread (determined by IC) from a igraph graph
    using the Cost Effective Lazy Forward Algorithm, a.k.a Lazy Greedy Algorithm.
    """
    start_time = time.time()
    scenerios = generate_sensor_scenerios(n_scenerios, min_sensor_count, max_sensor_count)

    # find the first node with greedy algorithm:
    # TODO:: python's heap is a min-heap, thus
    # TODO:: we negate the spread to get the node
    # TODO:: with the maximum spread when popping from the heap
    print("started")
    gains = []
    for node in set(graph.nodes):
        delay = compute_delay(graph, [node], scenerios)
        delay_gain = sim.upper_bound_delay - delay
        heapq.heappush(gains, (-delay_gain, node, delay))

    # we pop the heap to get the node with the best spread,
    # TODO:: when storing the spread to negate it again to store the actual spread
    delay_gain, node, delay = heapq.heappop(gains)
    delay_gain = -delay_gain
    gateways = [node]
    delay_gains = [delay_gain]
    delays = [delay]

    # record the number of times the spread is computed
    lookups = [graph.number_of_nodes()]
    elapsed = [round(time.time() - start_time, 3)]

    for _ in range(budget - 1):
        node_lookup = 0
        matched = False

        while not matched:
            node_lookup += 1

            # TODO:: here we need to compute the marginal gain of adding the current node
            # to the solution, instead of just the gain, i.e. we need to subtract
            # the spread without adding the current node
            _, current_node, _ = heapq.heappop(gains)
            delay = compute_delay(graph, gateways + [current_node], scenerios)
            new_delay_gain = (sim.upper_bound_delay - delay)- delay_gain

            # check if the previous top node stayed on the top after pushing
            # the marginal gain to the heap
            heapq.heappush(gains, (-new_delay_gain, current_node, delay))
            matched = (gains[0][1] == current_node)

        # spread stores the cumulative spread
        new_delay_gain, node, delay = heapq.heappop(gains)
        delay_gain -= new_delay_gain
        gateways.append(node)
        delay_gains.append(delay_gain)
        delays.append(delay)
        lookups.append(node_lookup)

        elapse = round(time.time() - start_time, 3)
        elapsed.append(elapse)

    return gateways, delays, elapsed, lookups # delay_gains